# The ensemble approach is to take the data, and train each of the N classifiers on a different random arrangement of 6 landmarks. 

In [ ]:
# first get all X and y data from the all_points_folds
import numpy as np 
import pickle, random 
import cv2

X, y = [], []

for file in os.listdir("all_points_folds"): 
    with open(f"all_points_folds/{file}", 'rb') as f: 
        X_y = pickle.load(f)
        X.append(X_y[0])
        y.append(X_y[1])

X = np.concatenate([X_i for X_i in X])
y = np.concatenate([y_i for y_i in y]) 

# shuffle data just to be sure
N = np.random.permutation(X.shape[0])
X, y = X[N], y[N]

In [ ]:
# create the randomly generated hand arrangments for 200 models

hand_arrangements = [] # (200, 6) matrix

for arrangement in range(200): 
    hand_arrangements.append(random.sample(list(range(0, 21)), 6))


# create a function that will give X_train, X_test, y_train, y_test based on 
# the hand locations 

def generate_dataset(hand_arrangement, X, y): 
    columns_of_landmarks = [] 
    for landmark_no in hand_arrangement: 
        for _ in [0, 21, 42, 63, 84, 105, 126]: 
            